In [ ]:

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


In [ ]:
from json import *
import numpy as np
with open('task3_train.txt','r')as f:
    task3_train=f.read()
task3_train=JSONDecoder().decode(task3_train)

with open('task3_hash.txt','r')as f:
    task3_hash=f.read()
task3_hash=JSONDecoder().decode(task3_hash)

with open('task3_trainB.txt','r')as f:
    task3_trainB=f.read()
task3_trainB=JSONDecoder().decode(task3_trainB)

with open('task3_test.txt','r')as f:
    task3_test=f.read()
task3_test=JSONDecoder().decode(task3_test)

with open('task3_train_feature.txt','r')as f:
    train_feature=f.read()
train_feature=JSONDecoder().decode(train_feature)

with open('task3_test_feature.txt','r')as f:
    test_feature=f.read()
test_feature=JSONDecoder().decode(test_feature)


In [ ]:
word_dict={'PADDING':[0,9999],'UNK':[1,9999]}
pos_dict={'PADDING':0,'UNK':1}

for sent in task3_train:
    for word in sent['word']:
        if not str(word) in word_dict:
            word_dict[str(word)]=[len(word_dict),1]
        else:
            word_dict[str(word)][1]+=1
    for pos in sent['pos']:
        if not pos in pos_dict:
            pos_dict[str(pos)]=len(pos_dict)
            
for sent in task3_hash:
    for word in sent['word']:
        if not  str(word) in word_dict:
            word_dict[str(word)]=[len(word_dict),1]
        else:
            word_dict[str(word)][1]+=1
    for pos in sent['pos']:
        if not pos in pos_dict:
            pos_dict[str(pos)]=len(pos_dict)
            
            
for sent in task3_test:
    for word in sent['word']:
        if not  str(word) in word_dict:
            word_dict[str(word)]=[len(word_dict),1]
        else:
            word_dict[str(word)][1]+=1
    for pos in sent['pos']:
        if not pos in pos_dict:
            pos_dict[str(pos)]=len(pos_dict)
                         

In [ ]:
print(len(word_dict))

In [ ]:
embdict=dict()

with open('/home/wuch/word2vec_twitter_model.bin','rb')as f:
    header = f.readline()
    vocab_size, layer1_size = map(int, header.split())
    binary_len = np.dtype('float32').itemsize * layer1_size
    for line in range(vocab_size):
        word = []
        while True:
            ch = f.read(1).decode(errors='ignore')
            if ch ==' ':
                word = ''.join(word)
                break
            if ch != '\n':
                word.append(ch)
        if len(word) != 0:
            tp= np.fromstring(f.read(binary_len), dtype='float32')
            if word in word_dict:
                embdict[str(word)]=tp.tolist()

        else:
            f.read(binary_len)


In [ ]:
embdict2=dict()

#model_swm_300-6-10-low.w2v
with open('/data1/wuch/model_swm_300-6-10-low.w2v','r',encoding='utf-8',errors='ignore')as f:
    header = f.readline()
    vocab_size, layer1_size = map(int, header.split())
    binary_len = np.dtype('float32').itemsize * layer1_size
    for line in range(vocab_size):
        k = f.readline().split()
        word=k[0]
        if len(word) != 0:
            tp=[float(x) for x in k[1:]]
            #tp= np.fromstring(' '.join(k[1:]), dtype='float32')
            if word in word_dict:
                embdict2[str(word)]=tp
        else:
            continue



In [ ]:
from numpy.linalg import cholesky
print(len(embdict),len(word_dict))
print(len(word_dict))
lister=[0]*len(word_dict)
xp=np.zeros(400,dtype='float32')

cand=[]
for i in embdict.keys():
    lister[word_dict[i][0]]=np.array(embdict[i],dtype='float32')
    cand.append(lister[word_dict[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(lister)):
    if type(lister[i])==int:
        lister[i]=np.reshape(norm, 400)
lister[0]=np.zeros(400,dtype='float32')
lister=np.array(lister,dtype='float32')
print(lister.shape)


In [ ]:
from numpy.linalg import cholesky
print(len(embdict2),len(word_dict))
print(len(word_dict))
lister2=[0]*len(word_dict)
xp=np.zeros(300,dtype='float32')

cand=[]
for i in embdict2.keys():
    lister2[word_dict[i][0]]=np.array(embdict2[i],dtype='float32')
    cand.append(lister2[word_dict[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(lister2)):
    if type(lister2[i])==int:
        lister2[i]=np.reshape(norm, 300)
lister2[0]=np.zeros(300,dtype='float32')
lister2=np.array(lister2,dtype='float32')
print(lister2.shape)


In [ ]:
import pickle


In [ ]:
weight_num=[1923.,1390.,316.,205.]

In [ ]:
train_word=[]
train_pos=[]
train_class=[]
train_label=[]
train_weight=[]
train_index=[]
dev_word=[]
dev_pos=[]
dev_class=[]
dev_label=[]

test_word=[]
test_pos=[]
test_class=[]
test_label=[]
min_freq=2
maxlen=50

for sent in task3_hash:

    
    thash=[]
    for j in range(len(sent['word'])):
        if sent['word'][j] in embdict  or str(sent['word'][j]) in embdict2:
          
            if str(sent['word'][j])=='#irony':
                thash=[1,0,0,0]
            elif str(sent['word'][j])=='#not':
                thash=[0,1,0,0]
            elif str(sent['word'][j])=='#sarcasm' or str(sent['word'][j])=='#sarcastic':
                thash=[0,0,1,0]
            else:
                thash=[0,0,0,1]
    train_index.append(thash)
weight=[1.0,1.2,1.5,2.0] 
for sent in task3_train:  
    train_label.append(float(sent['label']))
for sent in task3_trainB:  
    tw=[]
    tp=[]
    for j in range(len(sent['word'])):
        if sent['word'][j] in embdict  or str(sent['word'][j]) in embdict2:
            
            if str(sent['word'][j])=='USERNAME':
                
                tw.append(word_dict['USERNAME'][0])
                poss=[0]*len(pos_dict)
                poss[pos_dict['@']]=1
                tp.append(poss)
            
            else:
                tw.append(word_dict[str(sent['word'][j])][0])
                poss=[0]*len(pos_dict)
                poss[pos_dict[str(sent['pos'][j])]]=1
                tp.append(poss)
    train_weight.append(np.sqrt(weight_num[0]/weight_num[int(sent['label'])]))
    if len(tw)>maxlen:
        maxlen=len(tw)
    tw=tw+[0]*(maxlen-len(tw))
    poslist=[0]*len(pos_dict)
    tp=tp+[poslist]*(maxlen-len(tp))
    train_word.append(tw)
    
    train_pos.append(tp)
    #train_label.append(float(sent['label']))
train_labelB=[]
for sent in task3_trainB:
    if int(sent['label']==0):
        train_labelB.append([1,0,0,0])
    if int(sent['label']==1):
        train_labelB.append([0,1,0,0])
    if int(sent['label']==2):
        train_labelB.append([0,0,1,0])
    if int(sent['label']==3):
        train_labelB.append([0,0,0,1])
for sent in task3_test:
    tw=[]
    tp=[]

    for j in range(len(sent['word'])):
        if sent['word'][j] in embdict  or str(sent['word'][j]) in embdict2:
            if str(sent['word'][j])=='USERNAME':
                
                tw.append(word_dict['USERNAME'][0])
                poss=[0]*len(pos_dict)
                poss[pos_dict['@']]=1
                tp.append(poss)
            
            else:
                tw.append(word_dict[str(sent['word'][j])][0])
                poss=[0]*len(pos_dict)
                poss[pos_dict[str(sent['pos'][j])]]=1
                tp.append(poss)

    if len(tw)>maxlen:
        maxlen=len(tw)
    tw=tw+[0]*(maxlen-len(tw))
    poslist=[0]*len(pos_dict)
    tp=tp+[poslist]*(maxlen-len(tp))
    test_word.append(tw)
    test_pos.append(tp)
    test_label.append(float(sent['label']))
print(maxlen)


In [ ]:
DATASET_FP = "./SemEval2018-T3_gold_test_taskB_emoji.txt"


corpus3, y3 = parse_dataset(DATASET_FP)

In [ ]:
import random
import random
import numpy as np
from sklearn import metrics
import sklearn
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf  
import keras
from keras import activations 
from keras import initializers 
from keras import constraints 
from keras import regularizers
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist
from keras.layers import GRU,Bidirectional,Reshape,Multiply,Conv1D,MaxPooling1D,GlobalMaxPooling1D,GlobalAveragePooling1D,concatenate
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout,multiply, Input,Activation,wrappers,Merge,merge,core,advanced_activations,Conv2D,MaxPooling2D
from keras.layers import LSTM,Average,average,BatchNormalization
from keras.layers import Masking, Embedding,Flatten,Multiply,RepeatVector,Permute,TimeDistributed,add,GlobalAveragePooling2D
from keras.regularizers import l2,l1
from keras.constraints import maxnorm
from keras.callbacks import ModelCheckpoint,EarlyStopping
all_score=[]
all_pred=[]
for fold in range(1):
    
    test_word=np.array(test_word,dtype='float32')
    test_pos=np.array(test_pos,dtype='float32')
    test_feature=np.array(test_feature,dtype='float32')
    
    split_train_word=np.array(train_word)
    split_train_pos=np.array(train_pos)
    split_train_feature=np.array(train_feature)
    split_train_label=np.array(train_label)
    split_train_labelB=np.array(train_labelB)
    split_train_index=np.array(train_index)
    split_train_weight=np.array(train_weight)

    ensen=10
    for wei in range(ensen):
        
        
        sequenceLength =50
        fmin=0
        batch_size=64

        x_train = Input(batch_shape=(None,sequenceLength),dtype='float32')
        x_train1 = Input(batch_shape=(None,sequenceLength,len(pos_dict)),dtype='float32')
        x_train3 = Input(batch_shape=(None,149),dtype='float32')
        emb_drop=0.2
        hid1=500
        
        e0=Embedding(len(word_dict), 400,weights=[lister],input_length=sequenceLength,mask_zero=True)(x_train)
        d0=Dropout(emb_drop)(e0)
        e1=Embedding(len(word_dict), 300,weights=[lister2],input_length=sequenceLength,mask_zero=True)(x_train)
        d1=Dropout(emb_drop)(e1)
        #e1=Embedding(100, len(pos_dict),input_length=sequenceLength,mask_zero=True)(x_train1)
        #d1=Dropout(emb_drop)(e1)
        
        h_merge1=merge([d0,d1,x_train1], mode='concat',concat_axis=2)
        #fa=Reshape((sequenceLength,400+len(pos_dict),1))(h_merge1)
        flat_beta=Bidirectional(GRU(300,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))(h_merge1)
        flat_beta2=Bidirectional(GRU(300,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))(flat_beta)
        h_merge3=concatenate([flat_beta,flat_beta2])
        #
        flat_beta3=Bidirectional(GRU(300,return_sequences=False,dropout=0.4,recurrent_dropout=0.4))(h_merge3)
        
        cr2=Dense(200,activation='relu')(flat_beta3)
        cr4=Dense(4,activation='softmax')(cr2)
        
    
        
        h_merge6=concatenate([cr2,cr4,flat_beta3])
        cr1=Dense(200,activation='relu')(h_merge6)
        
        cr3=Dense(4,activation='softmax',name='taskb')(cr1)
        
        Acr1=Dense(200,activation='relu')(h_merge6)
        
        Acr3=Dense(1,activation='sigmoid')(Acr1)
        
        model=Model([x_train,x_train1,x_train3],[cr3,cr4,Acr3])
        
        model.compile('rmsprop', loss=['categorical_crossentropy','categorical_crossentropy','binary_crossentropy'],
                      metrics=['acc','acc'],loss_weights=[0.5,0.5,1.])
        
        for _ in range(3):
            model.fit([split_train_word,split_train_pos,split_train_feature],[split_train_labelB,split_train_index,split_train_label],
                      batch_size=batch_size, nb_epoch=1,shuffle=True,sample_weight={'taskb':split_train_weight})


            classes=np.argmax(model.predict([test_word,test_pos,test_feature])[0],axis=1)
            classes2=np.round(model.predict([test_word,test_pos,test_feature])[2])
            score = sklearn.metrics.f1_score(y2, classes2, pos_label=1)
            score2 = sklearn.metrics.precision_score(y2, classes2, pos_label=1)
            score3 = sklearn.metrics.recall_score(y2, classes2, pos_label=1)
            score4 = sklearn.metrics.classification_report(y3, classes, digits=4)

            print ("F1-score Task A",score2,score3,score)
            print ("F1-score Task B",score4)
